In [4]:
import os
import pickle
import requests
import numpy as np
import random
import math

In [15]:
total_num_examples = 4000 #1000 #20000 #5000 #2000
num_digits = 3


In [16]:
num_digit_2 = int(900*total_num_examples/10000)
num_digit_list = [100, num_digit_2, total_num_examples - 100 - num_digit_2]

target_num_carry_examples = math.ceil(total_num_examples / (num_digits+1))
num_carry_list = [0 for i in range(num_digits+1)]

input_file_path = f'add_examples_{total_num_examples}.txt'
output_file_path = f'add_examples_{total_num_examples}_shuffled.txt'

In [17]:
def numCarryOps(a, b):
    a,b=int(a),int(b)
    def digitSum(n):
        return sum(map(int,str(n)))
    # assert(a >= 0); assert(b >= 0);
    return int((digitSum(a) + digitSum(b) - digitSum(a+b)) / 9)

In [18]:
# print('making examples of a+b=c')
if not os.path.exists(input_file_path):
    with open(input_file_path, 'w') as f:
        num_example = 0
        for num_digit in range(1, num_digits+1):
            num_digit_example = 0
            print(num_digit, num_digit_list[num_digit-1], num_example, num_carry_list)
            while num_digit_example < num_digit_list[num_digit-1] and num_example < total_num_examples:
            # generate a random number between 0 and 10^(i+1) - 1
                a = random.randint(0, 10**(num_digit) - 1)
                b = random.randint(0, 10**(num_digit) - 1)
                c = a + b

                # count number of carries in c
                num_carry = numCarryOps(a, b)
                if num_carry_list[num_carry] < target_num_carry_examples:
                    # write the example to file
                    f.write(f'${a}+{b}={c}$\n')
                    # increment num_carry_list[num_carry]
                    num_carry_list[num_carry] += 1
                    num_digit_example += 1
                    num_example += 1
                else:
                    continue


    with open(input_file_path, 'r') as f:
            lines = f.readlines()
            random.shuffle(lines)
            with open(output_file_path, 'w') as f:
                f.writelines(lines)

1 100 0 [0, 0, 0, 0]
2 360 100 [60, 40, 0, 0]
3 3540 460 [173, 197, 90, 0]


In [19]:
with open(output_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
# n = len(data) # 130,023
# train_data = data[:int(n*0.9)]
# val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(data)
# val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
# print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
# val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(f'train_{total_num_examples}.bin')
# val_ids.tofile(f'val_{total_num_examples}.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(f'meta.pkl', 'wb') as f:
    pickle.dump(meta, f)


# length of dataset in characters: 142,699
# all the unique characters: 
# $+0123456789=
# vocab size: 14
# train has 128,429 tokens
# val has 14,270 tokens

length of dataset in characters: 55,585
all the unique characters: 
$+0123456789=
vocab size: 14
train has 55,585 tokens
